<a href="https://colab.research.google.com/github/engsoft7/GTPdoagro/blob/main/Projeto_1_Chatbot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Especificamente:
Modelo: LLaMA-3.1-8B-Instant
Infraestrutura: GROQ Cloud
Acesso: API externa (cloud)
Autenticação: API Key (GROQ_API_KEY)

1. Instalar depenências

In [ ]:
!pip install --quiet groq sentence-transformers


2) Chave da API (Colab Secrets)

In [ ]:
GROQ_API_KEY = 'KEY'


In [ ]:
from google.colab import userdata

GROQ_KEY = userdata.get("KEY")
print("Groq key OK:", bool(GROQ_KEY))


Groq key OK: True


3) Imports e cliente GROQ

In [ ]:
from groq import Groq
from dataclasses import dataclass
import json
import re

client = Groq(api_key=GROQ_KEY)


4) Prompt Engineering (templates)

In [ ]:
@dataclass
class PromptTemplates:
    system: str
    instructions: str
    response_format: str
    persona: str

templates = PromptTemplates(
    system = (
        "Você é um assistente especialista de domínio. "
        "Sempre responda em português. "
        "Se não souber, diga claramente que não sabe."
    ),

    instructions = (
        "Use SOMENTE o DOMAIN_KB para fatos técnicos. "
        "Não invente dados. "
        "Se estiver incerto, marque confidence como unknown."
    ),

    response_format = (
        "{\n"
        "  \"answer\": \"resposta objetiva\",\n"
        "  \"sources\": [\"kb_id\"],\n"
        "  \"confidence\": \"low|medium|high|unknown\",\n"
        "  \"notes\": \"opcional\"\n"
        "}\n"
        "Responda SOMENTE nesse JSON."
    ),

    persona = "Você é o ExpertBot: direto, técnico e sem enrolação."
)


5) Conhecimento de domínio (Domain-Specific LLM)

In [ ]:
domain_kb = [
    {
        "id": "kb1",
        "text": "A análise de solo avalia pH, fósforo, potássio e matéria orgânica."
    },
    {
        "id": "kb2",
        "text": "O pH ideal do solo para soja varia entre 5.5 e 6.8."
    },
    {
        "id": "kb3",
        "text": "A análise de solo deve ser feita antes do plantio para correção de nutrientes."
    }
]


6) Memória curta (short-term memory)

In [ ]:
class ShortTermMemory:
    def __init__(self, max_turns=5):
        self.max_turns = max_turns
        self.turns = []

    def add(self, user, assistant):
        self.turns.append((user, assistant))
        if len(self.turns) > self.max_turns:
            self.turns.pop(0)

    def format(self):
        return "\n".join(
            [f"Usuário: {u}\nAssistente: {a}" for u, a in self.turns]
        )

memory = ShortTermMemory()


7) Montagem do prompt completo

In [ ]:
def build_prompt(user_query):
    kb_text = "\n".join([f"[{d['id']}] {d['text']}" for d in domain_kb])
    mem = memory.format()

    system = (
        templates.system + "\n" +
        templates.persona + "\n" +
        templates.instructions
    )

    user = (
        f"DOMAIN_KB:\n{kb_text}\n\n"
        f"MEMORIA:\n{mem}\n\n"
        f"PERGUNTA:\n{user_query}\n\n"
        f"FORMATO_RESPOSTA:\n{templates.response_format}"
    )

    return system, user


8) Chamada ao modelo GROQ

In [ ]:
def call_llm_groq(user_query, model="llama-3.1-8b-instant"):
    system, user = build_prompt(user_query)

    response = client.chat.completions.create(
        model=model,
        messages=[
            {"role": "system", "content": system},
            {"role": "user", "content": user}
        ],
        temperature=0.0,
        max_tokens=400
    )

    return response.choices[0].message.content


9) Validação de formato (JSON obrigatório)

In [ ]:
def validate_response(text):
    try:
        json_text = re.search(r"\{.*\}", text, re.DOTALL).group()
        data = json.loads(json_text)

        for key in ["answer", "sources", "confidence"]:
            if key not in data:
                raise ValueError(f"Campo ausente: {key}")

        return data

    except Exception as e:
        return {
            "answer": "Erro de validação",
            "sources": [],
            "confidence": "unknown",
            "notes": str(e)
        }


10) Loop do chatbot (funcional)

In [ ]:
print("Chat iniciado (digite 'sair' para encerrar)\n")

while True:
    pergunta = input("Você: ")

    if pergunta.lower() in ["sair", "exit", "quit"]:
        break

    raw = call_llm_groq(pergunta)
    result = validate_response(raw)

    print("\nResposta:")
    print(json.dumps(result, indent=2, ensure_ascii=False))

    memory.add(pergunta, result["answer"])


Chat iniciado (digite 'sair' para encerrar)


Resposta:
{
  "answer": "Olá, posso ajudar com suas perguntas sobre agricultura ou outras áreas relacionadas.",
  "sources": [],
  "confidence": "high",
  "notes": ""
}

Resposta:
{
  "answer": "Para plantar alface, é recomendado seguir os seguintes passos: 1) escolher um local com solo fértil e bem drenado; 2) realizar a análise de solo para determinar a necessidade de correção de nutrientes (DOMAIN_KB: [kb3]); 3) aplicar adubo orgânico em uma porcentagem de 2 a 3% (DOMAIN_KB: [kb1]); 4) semear as sementes de alface em uma distância de 10 a 15 cm entre as plantas; 5) manter a irrigação regular e a temperatura entre 15°C e 25°C.",
  "sources": [
    "kb1",
    "kb3"
  ],
  "confidence": "high",
  "notes": "A alface é uma planta que cresce rapidamente e pode ser colhida em 20 a 30 dias após a semeadura."
}


✅ Chatbot:

✔ GROQ + LLaMA 3
✔ Prompt Engineering
✔ Domain-Specific LLM leve
✔ Memória curta
✔ Validação de saída
✔ Anti-alucinação por regra + KB
✔ Pronto para Colab

“O chatbot foi especializado no domínio de análise de solo e manejo inicial da cultura da soja, fornecendo respostas técnicas baseadas em conhecimento agronômico previamente incluído no prompt.”

Qual o teor ideal de sodio no solo?
O que é necessário para nutrir uma planta?
Quanto de aguá é necessário para irrigar uma rosa do deserto?
Como fazer uma aquaponia?
Porcentagem de adubo para uma planta

“O chatbot utiliza um modelo fundacional (LLaMA 3.1) acessado via API do GROQ, com especialização realizada por Prompt Engineering e injeção de conhecimento de domínio, sem fine-tuning.”